In [1]:
import pulp
import json
import networkx as nx
import matplotlib.pyplot as plt
import time

def solve_hub_problem(input_file):
    print(f"\n--- Loading and Processing Data for {input_file} ---")
    start_time = time.time()

    with open(input_file, 'r') as f:
        data = json.load(f)

    nodes = list(range(1, int(data['NodeNum']) + 1))
    n = len(nodes)

    flow = {(i+1, j+1): data['flow(wij)'][i][j] for i in range(n) for j in range(n)}
    cost = {(i+1, j+1): data['varCost(cij)'][i][j] for i in range(n) for j in range(n)}
    fix_cost = {k+1: data['fixCost(fk)'][k] for k in range(n)}
    capacity = {k+1: data['Cap(ckmax)'][k] for k in range(n)}
    alpha = data['alpha']
    O = {i: sum(flow[(i, j)] for j in nodes) for i in nodes}

    print(f"Data loaded in {time.time() - start_time:.2f} seconds.")
    print(f"Number of nodes: {n}")

    print("--- Setting up Optimization Model ---")
    model = pulp.LpProblem("Hub_Location_Problem", pulp.LpMinimize)

    Y = pulp.LpVariable.dicts("HubLink", [(k, l) for k in nodes for l in nodes if k < l], cat='Binary')
    Z = pulp.LpVariable.dicts("Allocation", [(i, k) for i in nodes for k in nodes], cat='Binary')
    X = pulp.LpVariable.dicts("Flow", [(i, k, l) for i in nodes for k in nodes for l in nodes if k != l], lowBound=0)

    fixed_cost = pulp.lpSum(fix_cost[k] * Z[(k, k)] for k in nodes)
    collection_cost = pulp.lpSum(cost[(i, k)] * O[i] * Z[(i, k)] for i in nodes for k in nodes)
    distribution_cost = pulp.lpSum(cost[(l, j)] * flow[(i, j)] * Z[(j, l)] for i in nodes for j in nodes for l in nodes)
    transfer_cost = pulp.lpSum(alpha * cost[(k, l)] * X[(i, k, l)] for i in nodes for k in nodes for l in nodes if k != l)
    model += fixed_cost + collection_cost + distribution_cost + transfer_cost, "Total_Cost"

    print("Objective function defined.")
    print("--- Adding Constraints ---")

    print("Adding Constraint 1: Single Allocation...")
    for i in nodes:
        model += pulp.lpSum(Z[(i, k)] for k in nodes) == 1, f"SingleAlloc_{i}"

    print("Adding Constraint 2: Hub Link Logic...")
    for k, l in Y:
        model += Y[(k, l)] <= Z[(k, k)], f"HubLink_k_{k}_{l}"
        model += Y[(k, l)] <= Z[(l, l)], f"HubLink_l_{k}_{l}"

    print("Adding Constraint 3: Flow Routing...")
    for i in nodes:
        if O[i] > 0:
            for k, l in Y:
                model += X[(i, k, l)] <= O[i] * Y[(k, l)], f"FlowRoute_{i}_{k}_{l}"
                model += X[(i, l, k)] <= O[i] * Y[(k, l)], f"FlowRoute_{i}_{l}_{k}"

    print("Adding Constraint 4: Flow Conservation...")
    for i in nodes:
        for k in nodes:
            out_flow = pulp.lpSum(X[(i, k, l)] for l in nodes if l != k)
            in_flow = pulp.lpSum(X[(i, m, k)] for m in nodes if m != k)
            net = O[i] * Z[(i, k)] - pulp.lpSum(flow[(i, j)] * Z[(j, k)] for j in nodes)
            model += (out_flow - in_flow) == net, f"FlowCons_{i}_{k}"

    print("Adding Constraint 5: Hub Capacity...")
    for k in nodes:
        incoming = pulp.lpSum(X[(i, m, k)] for i in nodes for m in nodes if m != k)
        local = pulp.lpSum(O[i] * Z[(i, k)] for i in nodes)
        model += incoming + local <= capacity[k] * Z[(k, k)], f"Capacity_{k}"

    print("Adding Constraint 6: Tree Structure...")
    model += pulp.lpSum(Y[(k, l)] for k, l in Y) == pulp.lpSum(Z[(k, k)] for k in nodes) - 1, "TreeStructure"

    print("--- Solving the Model ---")
    solver = pulp.PULP_CBC_CMD(msg=True)  # Optional: add `timeLimit=3600`
    model.solve(solver)

    print(f"\n{'='*40}\nSolution for {input_file}:")
    print(f"Status: {pulp.LpStatus[model.status]}")
    if model.status in [pulp.LpStatusOptimal, pulp.LpStatusFeasible]:
        print(f"Total Cost: {pulp.value(model.objective):,.2f}")
        hubs = [k for k in nodes if pulp.value(Z[(k, k)]) > 0.99]
        print(f"Hubs: {hubs}")

        allocations = {i: k for i in nodes for k in nodes if pulp.value(Z[(i, k)]) > 0.99}
        print("Allocations:")
        for k in hubs:
            assigned = sorted([i for i in nodes if i != k and allocations[i] == k])
            print(f"  Hub {k}: {assigned}")

        links = [(k, l) for k, l in Y if pulp.value(Y[(k, l)]) > 0.99]
        print(f"Hub Links: {links}")

        try:
            print("--- Generating Network Visualization ---")
            G = nx.Graph()
            G.add_nodes_from(nodes)
            colors = ['red' if n in hubs else 'lightblue' for n in nodes]
            sizes = [600 if n in hubs else 300 for n in nodes]

            edges = []
            edge_colors = []
            edge_widths = []

            for k, l in links:
                edges.append((k, l))
                edge_colors.append('red')
                edge_widths.append(2.5)

            for i, k in allocations.items():
                if i != k:
                    edges.append((i, k))
                    edge_colors.append('gray')
                    edge_widths.append(1.0)

            G.add_edges_from(edges)
            pos = nx.kamada_kawai_layout(G)

            plt.figure(figsize=(15, 10))
            nx.draw(G, pos, node_color=colors, node_size=sizes, edge_color=edge_colors, width=edge_widths, with_labels=True)
            plt.title(f"Hub Network: {input_file} (Cost: {pulp.value(model.objective):,.2f})", fontsize=16)
            plt.show()
        except Exception as e:
            print(f"Plot error: {e}")
    else:
        print("No optimal solution found.")

    print(f"Total execution time: {time.time() - start_time:.2f} seconds.")
    return model


In [ ]:
model_large = solve_hub_problem('InputDataHubLargeInstance.json')



--- Loading and Processing Data for InputDataHubLargeInstance.json ---
Data loaded in 0.01 seconds.
Number of nodes: 30
--- Setting up Optimization Model ---
Objective function defined.
--- Adding Constraints ---
Adding Constraint 1: Single Allocation...
Adding Constraint 2: Hub Link Logic...
Adding Constraint 3: Flow Routing...
Adding Constraint 4: Flow Conservation...
Adding Constraint 5: Hub Capacity...
Adding Constraint 6: Tree Structure...
--- Solving the Model ---
